# First thing to do, make a copy of this notebook under File->Make a Copy with some intelligent naming convention.

---

# Set your observing times here.

## NB: This will be a broader window that you'll actually use. The final output will give you orbit-by-orbit pointing information. So you can use a subset of the orbits that are chosen here.

In [40]:
from datetime import datetime
import numpy as np
tstart = '2018-07-19T10:00:00'
tend = '2018-07-20T10:00:00'
 
# Turn these into datetime objects

tstart = datetime.strptime(tstart, '%Y-%m-%dT%H:%M:%S')
tend = datetime.strptime(tend, '%Y-%m-%dT%H:%M:%S')                

---
# Step 1, get the nominal RA/Dec Position for the Moon at the start.

## We use this to determine what the occultation times are.

In [41]:
from skyfield.api import Loader
from astropy.time import Time
import astropy.units as u

load = Loader('../../data')

ts = load.timescale()
planets = load('de436.bsp')

astro_time = Time(tstart)
t = ts.from_astropy(astro_time)
moon, earth = planets['moon'], planets['earth']
astrometric = earth.at(t).observe(moon)
ra, dec, distance = astrometric.radec()


occstring = "./occ {0:0.4f} {1:0.4f} Latest_TLE.txt {2}:{3}:00:00:00 {4}:{5}:00:00:00 lunar_{3}_{5}.occ".format(
      ra.to(u.deg).value, dec.to(u.deg).value,
      tstart.timetuple().tm_year, tstart.timetuple().tm_yday, 
      tend.timetuple().tm_year, tend.timetuple().tm_yday)


outfile = '../orbit_engine/lunar_{0}to{1}.sh'.format(tstart.timetuple().tm_yday, tend.timetuple().tm_yday)
print(outfile)

f = open(outfile, 'w')
f.write(occstring)
f.close()


import os
import stat

st = os.stat(outfile)
os.chmod(outfile, st.st_mode | stat.S_IEXEC)



[#################################] 100% deltat.preds


../orbit_engine/lunar_200to201.sh


---
# Step 2: Go run the code that figures out the unocculted periods for the RA/Dec and the date range reported above.

This works on lif. There are example shell scripts in the ../orbit_engine directory that use the version that Karl already compiled for the nuops users.

First, get the latest TLE archive:

`./get_latest_TLE.sh`

Run the script that was produced above.


# Step 3: Initialize your libraries and parse the resulting occultation file:


In [3]:
from nustar_planning import io
occfile= "../orbit_engine/lunar_{0}_{1}.occ".format(tstart.timetuple().tm_yday,tend.timetuple().tm_yday )

#print(occfile)
orbits = io.parse_occ(occfile)

# NB: The "head" command here only shows the first couple of rows. Do a "print(orbits)" to see them all.
orbits.head()


,visible,occulted
0,2017-10-05 01:27:57,2017-10-05 02:23:46
1,2017-10-05 03:04:37,2017-10-05 04:00:25
2,2017-10-05 04:41:16,2017-10-05 05:37:05
3,2017-10-05 06:17:55,2017-10-05 07:13:44
4,2017-10-05 07:54:35,2017-10-05 08:50:23


In [4]:
import nustar_planning
import imp
imp.reload(nustar_planning)


<module 'nustar_planning' from '/Users/bwgref/science/local/anaconda/lib/python3.5/site-packages/nustar_planning-0.2.dev73-py3.5.egg/nustar_planning/__init__.py'>

In [36]:
# 6 arcminutes maximum drift (default)

nustar_planning.lunar_planning.position_shift(orbits[0:1], parallax_correction=True,
               outfile='drift_test.txt',
               dt = 5*u.s,pad_time = 5*u.min)


[#################################] 100% deltat.preds


In [37]:
from astropy.coordinates import SkyCoord

In [38]:
f = open('drift_test.txt', 'r')
times = []
sky_pos = []
for line in f:
    if line.startswith('A'):
        continue


    fields = line.split()
    this_time = datetime.strptime(fields[0], '%Y:%j:%H:%M:%S')

    times = np.append(times, this_time)
    
    this_skypos = SkyCoord(ra=fields[1], dec=fields[2], unit ='deg')
    sky_pos = np.append(sky_pos, this_skypos)
    
f.close()

In [39]:
good_exp = 0.
all_exp = 0.
for ind in range(len(times)):
    if ind == 0:
        continue
    dt = (times[ind] - times[ind-1]).total_seconds()
    dr = sky_pos[ind].separation(sky_pos[ind-1]).arcmin
    print(dt, dr)
    

235.0 8.169193212187796
935.0 8.23133421393569
185.0 8.074781471263107
155.0 8.273791552486983
145.0 8.111247337738043
135.0 8.29433747712782
135.0 8.267129350814725
135.0 8.03260858252313
135.0 8.136248831848992
145.0 8.11797452659852
160.0 8.106351868654194
190.0 8.029075013188795
270.0 8.070942939832577
660.0 8.158003531875288
